# Task 1 Multiplier

I used the circuit given by the authors of [1] to complete Task 1. They use a novel conditional adder in the multiplication circuit. I made sure that I use the exact steps given in [1] to make all the circuits.

In [1]:
import numpy as np
import qiskit as qs #Importing Qiskit
import qiskit.quantum_info as qi # Importing Quantum Information module for Statevector evolution

Below I have written a function to produce the conditional adder circuit taking in the number of qubits. I have used the exact steps as [1] to create the circuit.

In [2]:
def ctrl_adder(num_qubits=4):
    #Defining Registers 
    ctrl = qs.QuantumRegister(1,'ctrl')
    a_reg = qs.QuantumRegister(num_qubits+2,'a')
    b_reg = qs.QuantumRegister(num_qubits,'b')

    ctrl_add = qs.QuantumCircuit(ctrl,b_reg,a_reg)

    #Step 1 
    for i in range(1,num_qubits):
        ctrl_add.cx(a_reg[i],b_reg[i])

    #Step 2
    ## Sub-step 1
    ctrl_add.toffoli(ctrl,a_reg[num_qubits-1],a_reg[num_qubits])
    ## Sub-step 2
    for i in range(num_qubits-2,0,-1):
        ctrl_add.cx(a_reg[i],a_reg[i+1])

    #Step 3 
    for i in range(num_qubits-1):
        ctrl_add.toffoli(a_reg[i],b_reg[i],a_reg[i+1])

    #Step 4 
    ## Sub-step 1 
    ctrl_add.toffoli(a_reg[num_qubits-1],b_reg[num_qubits-1],a_reg[num_qubits+1])
    ## Sub-step 2 
    ctrl_add.toffoli(ctrl,a_reg[num_qubits+1],a_reg[num_qubits])
    ## Sub-step 3
    ctrl_add.toffoli(a_reg[num_qubits-1],b_reg[num_qubits-1],a_reg[num_qubits+1])
    ## Sub-step 4
    ctrl_add.toffoli(ctrl,a_reg[num_qubits-1],b_reg[num_qubits-1])

    #Step 5
    for i in range(num_qubits-2,-1,-1):
        ## Sub-step 1
        ctrl_add.toffoli(a_reg[i],b_reg[i],a_reg[i+1])
        ## Sub-step 2
        ctrl_add.toffoli(ctrl,a_reg[i],b_reg[i]) 

    #Step 6
    for i in range(1,num_qubits-1):
        ctrl_add.cx(a_reg[i],a_reg[i+1])

    #Step 7
    for i in range(1,num_qubits):
        ctrl_add.cx(a_reg[i],b_reg[i])

    return ctrl_add

The circuit below is the one shown in [1] where number of qubits($n$) is 6. Where the number of qubits($n$) refer to the qubits used to represent the numbers $a$ and $b$ in binary. Here binary encoding is used the represent the integers. The circuit below looks different compared to the one given in [1] because of the ordering of the qubits.

In [3]:
ctrl_adder(6).draw(fold=500)

ctrl: ───────────────────────────■──────────────────────────────────────────────────────■─────────■─────────■─────────■─────────■─────────■─────────■───────────────────────────
                                 │                                                      │         │         │         │         │         │       ┌─┴─┐                         
 b_0: ───────────────────────────┼────────────────────────■─────────────────────────────┼─────────┼─────────┼─────────┼─────────┼─────────┼────■──┤ X ├─────────────────────────
      ┌───┐                      │                        │                             │         │         │         │         │       ┌─┴─┐  │  └─┬─┘┌───┐                    
 b_1: ┤ X ├──────────────────────┼────────────────────────┼────■────────────────────────┼─────────┼─────────┼─────────┼─────────┼────■──┤ X ├──┼────┼──┤ X ├────────────────────
      └─┬─┘┌───┐                 │                        │    │                        │         │         │         │       ┌─┴─┐  │  └─┬─┘  │    │  └─┬─┘┌───┐               
 b_2: ──┼──┤ X ├─────────────────┼────────────────────────┼────┼────■───────────────────┼─────────┼─────────┼─────────┼────■──┤ X ├──┼────┼────┼────┼────┼──┤ X ├───────────────
        │  └─┬─┘┌───┐            │                        │    │    │                   │         │         │       ┌─┴─┐  │  └─┬─┘  │    │    │    │    │  └─┬─┘┌───┐          
 b_3: ──┼────┼──┤ X ├────────────┼────────────────────────┼────┼────┼────■──────────────┼─────────┼─────────┼────■──┤ X ├──┼────┼────┼────┼────┼────┼────┼────┼──┤ X ├──────────
        │    │  └─┬─┘┌───┐       │                        │    │    │    │              │         │       ┌─┴─┐  │  └─┬─┘  │    │    │    │    │    │    │    │  └─┬─┘┌───┐     
 b_4: ──┼────┼────┼──┤ X ├───────┼────────────────────────┼────┼────┼────┼────■─────────┼─────────┼────■──┤ X ├──┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──┤ X ├─────
        │    │    │  └─┬─┘┌───┐  │                        │    │    │    │    │         │       ┌─┴─┐  │  └─┬─┘  │    │    │    │    │    │    │    │    │    │    │  └─┬─┘┌───┐
 b_5: ──┼────┼────┼────┼──┤ X ├──┼────────────────────────┼────┼────┼────┼────┼────■────┼────■──┤ X ├──┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──┤ X ├
        │    │    │    │  └─┬─┘  │                        │    │    │    │    │    │    │    │  └─┬─┘  │    │    │    │    │    │    │    │    │    │    │    │    │    │  └─┬─┘
 a_0: ──┼────┼────┼────┼────┼────┼────────────────────────■────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────■────■────┼────┼────┼────┼────┼──
        │    │    │    │    │    │                      ┌─┴─┐  │    │    │    │    │    │    │    │    │    │    │    │    │    │    │    │  ┌─┴─┐       │    │    │    │    │  
 a_1: ──■────┼────┼────┼────┼────┼───────────────────■──┤ X ├──■────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────■────■──┤ X ├──■────■────┼────┼────┼────┼──
             │    │    │    │    │                 ┌─┴─┐└───┘┌─┴─┐  │    │    │    │    │    │    │    │    │    │    │    │    │  ┌─┴─┐     └───┘┌─┴─┐       │    │    │    │  
 a_2: ───────■────┼────┼────┼────┼──────────────■──┤ X ├─────┤ X ├──■────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────■────■──┤ X ├──────────┤ X ├──■────■────┼────┼────┼──
                  │    │    │    │            ┌─┴─┐└───┘     └───┘┌─┴─┐  │    │    │    │    │    │    │    │    │    │  ┌─┴─┐     └───┘          └───┘┌─┴─┐       │    │    │  
 a_3: ────────────■────┼────┼────┼─────────■──┤ X ├───────────────┤ X ├──■────┼────┼────┼────┼────┼────┼────┼────■────■──┤ X ├─────────────────────────┤ X ├──■────■────┼────┼──
                       │    │    │       ┌─┴─┐└───┘               └───┘┌─┴─┐  │    │    │    │    │    │    │  ┌─┴─┐     └───┘                         └───┘┌─┴─┐       │    │  
 a_4: ─────────────────■────┼────┼────■──┤ X ├─────────────────────────┤ X ├──■────┼────┼────┼────┼────■──

Below is function the maps the conditional adder to the multiplication circuit. The mapping is required to correctly map the inputs of conditional adder to the final multiplication circuit.

In [4]:
def mapping_ctomul(b,num_qubits):
    m = [b] 
    b_ind = list(range(b + 2*num_qubits,b + 3*num_qubits)) 
    a_ind = list(range(num_qubits,2*num_qubits))
    anc_ind = list(range(b + 3*num_qubits,b + 3*num_qubits+2))
    return  m + b_ind + a_ind + anc_ind

Defining a funciton to crate the multiplication circuit.

In [5]:
def mul_circ(num_qubits=4):
    #Defining Registers
    A_reg = qs.QuantumRegister(num_qubits,'a')
    B_reg = qs.QuantumRegister(num_qubits,'b')
    P_reg = qs.QuantumRegister(2*num_qubits+1,'p') # Register that will store the product

    qmul = qs.QuantumCircuit(B_reg,A_reg,P_reg)
    ctrl_add = ctrl_adder(num_qubits) # Calling the Conditonal adder circuit
    
    #Step 1
    for i in range(num_qubits):
        qmul.ccx(B_reg[0],A_reg[i],P_reg[i])
    qmul.barrier() # Adding a barrier 

    #Step 2 
    for i in range(1,num_qubits):
        qmul.compose(ctrl_add,mapping_ctomul(i,num_qubits),inplace=True) # This is where I add the conditional adders 
        qmul.barrier()    
    return qmul

The final circuit, I have added barries to distinguish between different conditional adders.

In [6]:
mul_circ(3).draw(fold=500)

░                                                                                       ░                                                                                       ░ 
b_0: ──■────■────■───░───────────────────────────────────────────────────────────────────────────────────────░───────────────────────────────────────────────────────────────────────────────────────░─
       │    │    │   ░                                                                                       ░                                                                                       ░ 
b_1: ──┼────┼────┼───░─────────────■────────────────────────■─────────■─────────■─────────■──────────────────░───────────────────────────────────────────────────────────────────────────────────────░─
       │    │    │   ░             │                        │         │         │         │                  ░                                                                                       ░ 
b_2: ──┼────┼────┼───░─────────────┼────────────────────────┼─────────┼─────────┼─────────┼──────────────────░─────────────■────────────────────────■─────────■─────────■─────────■──────────────────░─
       │    │    │   ░             │                        │         │         │         │                  ░             │                        │         │         │         │                  ░ 
a_0: ──■────┼────┼───░─────────────┼─────────■──────────────┼─────────┼─────────┼────■────■──────────────────░─────────────┼─────────■──────────────┼─────────┼─────────┼────■────■──────────────────░─
       │    │    │   ░             │       ┌─┴─┐            │         │         │  ┌─┴─┐  │                  ░             │       ┌─┴─┐            │         │         │  ┌─┴─┐  │                  ░ 
a_1: ──┼────■────┼───░───■─────────┼────■──┤ X ├──■─────────┼─────────┼────■────■──┤ X ├──┼────■────■────────░───■─────────┼────■──┤ X ├──■─────────┼─────────┼────■────■──┤ X ├──┼────■────■────────░─
       │    │    │   ░   │         │  ┌─┴─┐└─┬─┘┌─┴─┐       │         │  ┌─┴─┐  │  └─┬─┘  │  ┌─┴─┐  │        ░   │         │  ┌─┴─┐└─┬─┘┌─┴─┐       │         │  ┌─┴─┐  │  └─┬─┘  │  ┌─┴─┐  │        ░ 
a_2: ──┼────┼────■───░───┼────■────■──┤ X ├──┼──┤ X ├──■────┼────■────■──┤ X ├──┼────┼────┼──┤ X ├──┼────■───░───┼────■────■──┤ X ├──┼──┤ X ├──■────┼────■────■──┤ X ├──┼────┼────┼──┤ X ├──┼────■───░─
     ┌─┴─┐  │    │   ░   │    │    │  └───┘  │  └─┬─┘  │    │    │    │  └─┬─┘  │    │    │  └───┘  │    │   ░   │    │    │  └───┘  │  └─┬─┘  │    │    │    │  └─┬─┘  │    │    │  └───┘  │    │   ░ 
p_0: ┤ X ├──┼────┼───░───┼────┼────┼─────────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼─────────┼────┼───░───┼────┼────┼─────────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼─────────┼────┼───░─
     └───┘┌─┴─┐  │   ░   │    │    │         │    │    │    │    │    │    │    │    │  ┌─┴─┐       │    │   ░   │    │    │         │    │    │    │    │    │    │    │    │    │         │    │   ░ 
p_1: ─────┤ X ├──┼───░───┼────┼────┼─────────■────┼────┼────┼────┼────┼────┼────┼────■──┤ X ├───────┼────┼───░───┼────┼────┼─────────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼─────────┼────┼───░─
          └───┘┌─┴─┐ ░ ┌─┴─┐  │    │              │    │    │    │    │    │  ┌─┴─┐     └───┘     ┌─┴─┐  │   ░   │    │    │         │    │    │    │    │    │    │    │    │  ┌─┴─┐       │    │   ░ 
p_2: ──────────┤ X ├─░─┤ X ├──┼────┼──────────────■────┼────┼────┼────┼────■──┤ X ├───────────────┤ X ├──┼───░───┼────┼────┼─────────■────┼────┼────┼────┼────┼────┼────┼────■──┤ X ├───────┼────┼───░─
               └───┘ ░ └───┘┌─┴─┐  │                   │    │    │  ┌─┴─┐     └───┘               └───┘┌─┴─┐ ░ ┌─┴─┐  │    │              │    │    │    │    │    │  ┌─┴─┐     └───┘     ┌─┴─┐  │   ░ 
p_3: ────────────────░──────┤ X ├──┼───────────────────■────┼────■──┤ X ├──────────────────────────────┤ X ├─░─┤ X ├──┼────┼──────────────■────┼────┼────┼────┼────■──┤ X ├───────────────┤ X ├──┼───░─
                     

I simulate the circuit using a *statevector* method in *qiskit* called *evolve*. 

In [58]:
def multiplier(a,b,num_qubits=4):
    if max(a,b) > 2**num_qubits-1:
        print('Please enter a number which can be represented by',num_qubits,'qubits or alternatively you can increase the number of qubits.')
        return None
        
    qmul = mul_circ(num_qubits)
    print(qmul.draw(fold=200))
    A_reg_sv = qi.Statevector.from_int(a,2**num_qubits) # Encoding integers into qubits. Basis Encoding is used throughout.
    B_reg_sv = qi.Statevector.from_int(b,2**num_qubits)
    P_reg_sv = qi.Statevector.from_int(0,2**(2*num_qubits+1))

    initial_state = A_reg_sv.tensor(B_reg_sv)
    initial_state = P_reg_sv.tensor(initial_state)      # Creating the initial state by tensor products.

    final_state = initial_state.evolve(qmul)
    final_state = qi.partial_trace(final_state,list(range(2*num_qubits))+[4*num_qubits]) # 

    s = final_state.probabilities_dict(decimals=4)

    for k,v in s.items():
        product = int(k, 2) 
    return product

In [8]:
multiplier(51,2,5)

Please enter a number which can be represented by 5 qubits or alternatively you can increase the number of qubits.


In [61]:
multiplier(3,2,3)

                     ░                                                                                       ░                                                                                       ░ 
b_0: ──■────■────■───░───────────────────────────────────────────────────────────────────────────────────────░───────────────────────────────────────────────────────────────────────────────────────░─
       │    │    │   ░                                                                                       ░                                                                                       ░ 
b_1: ──┼────┼────┼───░─────────────■────────────────────────■─────────■─────────■─────────■──────────────────░───────────────────────────────────────────────────────────────────────────────────────░─
       │    │    │   ░             │                        │         │         │         │                  ░                                                                                       ░ 


6

In [59]:
multiplier(4,3,4)

                          ░                                                                                                                          ░                                              »
b_0: ──■────■────■────■───░──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────░──────────────────────────────────────────────»
       │    │    │    │   ░                                                                                                                          ░                                              »
b_1: ──┼────┼────┼────┼───░──────────────────■──────────────────────────────────■─────────■─────────■─────────■─────────■────────────────────────────░──────────────────────────────────────────────»
       │    │    │    │   ░                  │                                  │         │         │         │         │                            ░                                              »
b_2: ──┼──

12

## *References*
[1] T-count Optimized Design of Quantum Integer Multiplication. https://arxiv.org/pdf/1706.05113.pdf 